In [1]:
import numpy as np
import pandas as pd

In [2]:
grades_group_dict = {
    "stem": ["MTK", "BIO", "FIS", "KIM", "MTK_P", "GEO"],
    "language": ["IND", "ING", "SIND", "SING", "JER", "ANTR", "SUN"],
    "business": ["MTK", "MTK_P", "EKO", "IND", "ING"],
    "arts": ["SBK", "PKR"],
    "social": ["SOS", "SEJ", "SI"],
    "others": ["PKN", "PJK"]
}

def group_category(row):
    if row["mapel"] in grades_group_dict["stem"]:
        return "stem"
    elif row["mapel"] in grades_group_dict["language"]:
        return "language"
    elif row["mapel"] in grades_group_dict["business"]:
        return "business"
    elif row["mapel"] in grades_group_dict["arts"]:
        return "arts"
    elif row["mapel"] in grades_group_dict["social"]:
        return "social"

    return "others"

In [3]:
def unpivot_semester_df(df_sem, semester):
    # unpivot data
    df_sem = df_sem.melt(id_vars=["no_urut"])

    # truncate mapel name
    df_sem["mapel"] = df_sem["variable"].apply(lambda x: x[5:])

    # map new mapel name
    mapel_map = {value: value for value in df_sem["mapel"].unique()}
    mapel_map["MTK.1"] = "MTK_P"

    # map lintas minat
    if rombel == "mipa":
        mapel_map["LM"] = "EKO"
    elif rombel == "ips":
        mapel_map["LM"] = "KIM"
    else:
        mapel_map["LM"] = "JAP"

    df_sem["mapel"] = df_sem["mapel"].map(mapel_map)

    # add category
    df_sem["kategori"] = df_sem.apply(group_category, axis=1)

    # drop variable column
    df_sem = df_sem.drop(columns=["variable"])

    # set semester
    df_sem = df_sem.assign(semester=semester)

    return df_sem.reset_index()

In [13]:
dataframes = []
for rombel in ["mipa", "ips", "bahasa"]:
    df = pd.read_excel(f"../dataset/snmptn_{rombel}.xlsx")
    grades_semesters = list(df.columns.values)
    for semesterColumn in [f"sem{i}" for i in range(1, 6)]:
        semester = int(semesterColumn[-1])
        startIndex = grades_semesters.index(f"{semesterColumn}_PAI")
        endIndex = startIndex + 15

        df_sem = pd.concat([df.iloc[:, 0], df.iloc[:, startIndex:endIndex]], axis=1)
        dataframes.append(unpivot_semester_df(df_sem, semester))

In [15]:
# concat all data
df_all = pd.concat(dataframes).reset_index()

# drop unnecessary data
df_all = df_all.drop(columns=["level_0", "index"])

# assign rombongan belajar
df_all = df_all.assign(rombel=rombel)

# rename columns
df_all = df_all.rename(columns={"no_urut": "siswa_id", "value": "nilai"})

# save to file
df_all.to_csv(f"../processed/sn_nilai_all.csv", index=False)

## GET MASUK/TIDAK

In [18]:
dataframes = []
for rombel in ["mipa", "ips", "bahasa"]:
    df = pd.read_excel(f"../dataset/snmptn_{rombel}.xlsx")
    dataframes.append(df[["no_urut", "masuk"]])

In [19]:
# concat all data
df_all = pd.concat(dataframes).reset_index()

# save to file
df_all.to_csv(f"../processed/sn_kelulusan_all.csv", index=False)